# Python实例练习：在线购物数据分析

分析目的：
* 1.统计购买商品数前十的国家。
* 2.统计交易额前十的国家。
* 3.统计2011年各个月份的交易额并分析。
* 4.计算2011年客单价（销售总额/顾客总数）
* 5.分析用户消费行为（消费次数、消费金额、购买数量等）

# 1.导入相关数据库

In [40]:
import numpy as np
import pandas as pd
import pandas_profiling
import plotly as py
import plotly.graph_objs as go
import cufflinks
from plotly.offline import iplot,init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# 2.导入数据并理解

In [4]:
data=pd.read_csv('online_retail_data.csv',encoding="ISO-8859-1",dtype={'CustomerID':str})

* 使用pandas_profiling对数据做初步探索

In [5]:
data.profile_report(title='online_retail_data')

从上述报告，可以得到以下信息：
* 1、**异常值**：Unitprice存在1个负值，需进一步查看原数据，看是否存在数据错误并删除。
* 2、**缺失值**：CustomerID和Description存在缺失值，CustomerID缺失率高达24.9%。
* 3、**数据转化**:InvoiceDate的数据类型应转变为日期类型。
* 4、**重复值**：InvoiceNo、StockCode、InvoiceDate存在少量重复值，在计算客单价、订单平均消费、日均消费等的时候，需要注意剔除重复。
* 5、**负值**：Quantity存在负值，说明存在部分退货订单，在计算销售额（不含退货）时，应注意剔除负值。

# 3.数据清洗

In [6]:
#检查异常值
data[data['UnitPrice']<0]
#通过Description了解这两笔交易是用于调整坏账
#暂不做处理

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom


In [7]:
# 剔除缺失值
df1=data.dropna(subset=['CustomerID'],how='any').copy()
# 分析目的包含客单价计算，故剔除CustomerID的缺失值

In [8]:
# 数据类型转化
df1['InvoiceDate']=pd.to_datetime(df1['InvoiceDate'],errors='coerce').dt.date
df1['InvoiceDate']=pd.to_datetime(df1['InvoiceDate'],errors='coerce')

# 4.数据分析和可视化

## 4.1、问题1：统计购买商品数前十的国家

In [9]:
Quantity_first_10=df1[df1['Quantity']>0].groupby('Country').sum().\
sort_values('Quantity',ascending=False).head(10)[['Quantity']]

In [10]:
#使用plotly和cufflinks绘制交互式图形
Quantity_first_10['Quantity'].iplot(kind='bar',yTitle='商品数', \
                                    linecolor='black', title='商品数前十的国家')

* 英国购买商品数远远大于其他国家，占总商品数的84.6%，可得知英国是该公司商品销售的主要市场。

## 4.2、问题2：统计交易额前十的国家

In [11]:
#新建变量Price=Quantity*UnitPrice
df1['Price']=df1.apply(lambda x:x[3]*x[5],axis=1)

In [12]:
Price_first_10=df1[df1['Quantity']>0].groupby('Country').sum().\
sort_values('Price',ascending=False).head(10)[['Price']]

In [13]:
#使用plotly和cufflinks绘制交互式图形
Price_first_10['Price'].iplot(kind='bar',yTitle='交易额', \
                                    linecolor='black', title='交易额前十的国家')

## 4.3、问题3：统计2011年各个月份的交易额并分析

In [14]:
#取出2011年的数据
df2=df1[df1['InvoiceDate'].dt.year==2011].copy()
#取出月份
df2['Month']=df2['InvoiceDate'].dt.month

In [15]:
Month_price=df2[df2['Quantity']>0].groupby('Month').sum().sort_values\
('Price',ascending=False)[['Price']]

In [16]:
#使用plotly和cufflinks绘制交互式图形
Month_price['Price'].iplot(kind='bar',yTitle='每月交易额', \
                                    linecolor='black', title='交易额排行')

* 交易额最高的是11月份，其次是10月份、9月份，说明这3个月是产品旺季。

## 4.4、计算2011年客单价（销售总额/顾客总数）

In [17]:
sumPrice=df2[df2['Price']>0]['Price'].sum()
count_CustomerID=df2[df2['Price']>0]['CustomerID'].drop_duplicates().count()

In [18]:
avg=sumPrice/count_CustomerID
print(avg)

1976.4621981512207


## 4.5、分析用户消费行为

In [19]:
#计算2011年用户消费次数、用户消费金额、用户购买产品数量
Customer=df2[df2['Quantity']>0].groupby('CustomerID').agg({'InvoiceNo':'nunique','Quantity':np.sum,'Price':np.sum})

In [39]:
print(Customer[['InvoiceNo','Quantity']].iplot(kind='box',secondary_y='Quantity',\
                                               yTitle='InvoiceNo',secondary_y_title='Quantity',title='箱线图'))
print(Customer[['Price']].iplot(kind='box',yTitle='Price',title='箱线图2'))

None


None


从箱线图中可知：
* 2011年客户平均消费4次，但大部分客户的消费频率较低。1/4的客户消费次数超过4次，消费次数最多高达188次，是产品的忠诚客户，可相应实施客户忠诚计划。
* 2011年客户购买数平均1154件。20%的客户购买数超过了平均数，并贡献了75%的购买量。对该部分大批量购买的客户，应采取相应的购买数量激励政策，以保持长期合作。
* 2011年客户平均消费金额为1976元，但只有25%的客户消费金额高于1614元，并贡献了79%的交易额，说明部分客户的消费金额较大，属于高价值客户，应重点关注。